In [ ]:
import os
import argparse
import pandas as pd
from ramanchada.classes import RamanChada, SpectrumGroup




def rr_processing(SOP,commit=True):
    SOP.normalize('minmax')
    if commit:
        SOP.commit("baseline")    
    SOP.peaks(fitmethod='voigt', x_min=400, x_max=600, show=False, interval_width=2)
    if commit:
        SOP.commit("peaks")
    return SOP.bands

def parse_raman(f_name):
    bands = None
    filename, file_extension = os.path.splitext(f_name)
    if file_extension==".xlsx":
        return None
    if file_extension!=".cha":     
        print("Parsing file: '%s' ..." % f_name, end="\n")
        try:
            SOP = RamanChada(f_name)
            bands = rr_processing(SOP)
            print(bands)
        except Exception as err:
            print(err)

    return {
        'Filename': filename + ".cha",
        'Bands': bands
    }



def process_file(f_name, recurse=False,callback=parse_raman):
    if os.path.isfile(f_name):
        return [callback(f_name)]
    elif os.path.isdir(f_name) and recurse:
        return [callback(os.path.join(root, f)) for (root, dirs, files) in os.walk(f_name) for f in files]
    else:
        return []

In [ ]:
partners = ["FraunhoferLBF","ELODIZ","TOPSOE","NTUA","CSIC_icp","CSIC_icv","FNMT_Madrid","FNMT_Burgos","Encapsulae"]    
instruments=[
{"brand": "Renishaw", "model" :	"Qontor" , "wavelength" : [405, 514, 785] },
{"brand": "Perkin Elmer", "model" :	"RamanStation 400F", "wavelength" : [785]},
{"brand": "WITEC", "model" :	"A300" , "wavelength" : [532]},
{"brand": "BWTEK", "model" :	"iRaman", "wavelength" : [535, 785] },
{"brand": "Zolix", "model" :	"Finder Edge", "wavelength" : [785] },
{"brand": "Horiba", "model" :	"LabRam HR", "wavelength" : [488] },
{"brand": "Horiba", "model" :	"LabRam 1" , "wavelength" : [532/633]},
{"brand": "BWTEK", "model" :	"iRaman-plus BWS465-785S", "wavelength" : [785] },
{"brand": "BWTEK", "model" :	"Nanoram NR2" , "wavelength" : [785]},
{"brand": "WITEC", "model" :	"Alpha 500" , "wavelength" : [532, 785]},
{"brand": "Renishaw", "model" :	"inVia Reflex" , "wavelength" : [532, 785]},
{"brand": "Horiba", "model" :	"LabRam IHR320" , "wavelength" : [785]},
{"brand": "BWTEK", "model" :	"iRaman" , "wavelength" : [532]},
{"brand": "Zolix", "model" :	"Finder Edge" , "wavelength" : [785]},
{"brand": "BWTEK", "model" :	"iRaman", "wavelength" : [532,785] }
]
brands=[]
for i in instruments:
    if i["brand"].upper() not in brands:
        brands.append(i["brand"].upper())
print(brands)

In [ ]:
path = "D:\\nina\OneDrive\WP 3 - TF3_RoundRobin\Round Robin\Data Raman\LBF"
path = "D:\\nina\DATA\Projects\CHARISMA\RR1"

r = process_file(path,recurse=True)
print(r)

In [ ]:
import h5pyd
import h5py
import h5pyd._apps.hsload as hsload
import h5pyd._apps.hsinfo as hsinfo
import h5pyd._apps.utillib as utillib
import h5pyd._apps.hsdel as hsdel
import uuid, h5py
from  tempfile import TemporaryFile

class Teleport:
    def __init__(self,cfg,bucket="charisma",owner=None,domain="/",brands=brands):
        self.cfg=cfg
        self.bucket=bucket
        self.set_owner(owner)
        self.set_brands(brands)

    def set_brands(self,brands):
        self.brands = brands
    
    def set_domain(self,domain="/RoundRobin1/"):
        self.domain=domain

    def set_owner(self,owner=None):
        if owner is None:
            self.owner=self.cfg["hs_username"]
        else:
            self.owner=owner

    def check_folder(self,domain="/Round_Robin_1/"):
        try:
            dir = h5pyd.Folder(domain, endpoint=self.cfg["hs_endpoint"], username=self.cfg["hs_username"], password=self.cfg["hs_password"], bucket=self.bucket, owner=self.owner)
        except IOError as err:
            dir = h5pyd.Folder(domain, mode='x',  endpoint=self.cfg["hs_endpoint"], username=self.cfg["hs_username"], password=self.cfg["hs_password"], bucket=self.bucket, owner=self.owner)
        return dir

    def delete_domain_recursive(self,fname):
        try:
            #print(fname)
            f = h5pyd.Folder(fname+"/")
            n = f._getSubdomains()
            if n>0:
                for s in f._subdomains:
                    #print(s["name"])
                    self.delete_domain_recursive(s["name"])
        except Exception as err:
            print(">>>",fname,err)
        try:
            #print("Deleting ",fname)    
            hsdel.deleteDomain(fname)
        except Exception as err:
            print(err)

#[Sample name]_[instrument & laser wavelength]_[optical path]_[power]_[timexavgs] 
    def parse_name(self,f_name):
        tags = os.path.basename(f_name).replace(".cha","").split("_") 
        timexavgs = tags[4].split("x")
        print(len(tags),tags)
        misc=None
        if len(tags)>5:
            misc = "_".join(tags[5:])
        instrument = tags[1][0:-3].upper();    
        instrument_brand=""
        instrument_model=""   
        for b in self.brands:
            if instrument.startswith(b):
                instrument_brand = b
                instrument_model = instrument.replace(b,"")
                break;  
        results = {
            "sample" : tags[0],
            "instrument" : instrument,
            "instrument_brand" : instrument_brand,
            "instrument_model" : instrument_model,
            "wavelength" : tags[1][-3:],
            "optical_path" : tags[2],
            "power" : tags[3],
            "time"  : timexavgs[0],
            "scans" : timexavgs[1]
        }
        if misc != None:
            results["misc"] = misc
        return results

    def cha2tmp(self,fname):
        tmpfile = TemporaryFile()
        tf = h5py.File(tmpfile, 'w')
        attributes = self.parse_name(fname)
        try:
            with h5py.File(fname, "r") as f:       
                _group = tf.create_group("root")  
                for item in attributes.items():
                    try:
                        _group.attrs.create(item[0],item[1])
                    except Exception as err:
                        print(item,err)   

                for key in f.keys():
                    _tmp=f[key]
                    _dataset = _group.create_dataset(key, data=_tmp)
                    for item in f[key].attrs.items():
                        _dataset.attrs.create(item[0],item[1])
        except Exception as err:
            print(">>",err)

        return tf 

              

    def chada2hsds(self,fname,delete=False):
        filename, file_extension = os.path.splitext(fname)
        dest=None
        if file_extension==".cha":     
            
            try:
                self.check_folder(self.domain);           
            except Exception as err:
                print(self.domain,self.owner,err)
            #print("Loading file: '%s' ..." % fname, end="\n")     

            fin=None
            fout=None       
            try:
                dest = "{}{}".format(self.domain,os.path.basename(fname))
                #.replace(".cha",""))
                
                if delete:
                    try:
                        print("Delete ",dest)
                        hsdel.deleteDomain(dest)
                    except Exception as err:
                        print(err)                      

                fin = self.cha2tmp(fname)
                fout = h5pyd.File(dest, "w", endpoint=self.cfg["hs_endpoint"], username=self.cfg["hs_username"], password=self.cfg["hs_password"], bucket=self.bucket, retries=3, owner=self.owner)
                utillib.load_file(fin, fout, verbose=False,dataload="ingest")
                return dest
            except Exception as err:
                print(err)
   
                



In [ ]:

fname="D:\\nina\\DATA\\Projects\\CHARISMA\\RR1\\LBF\\S0P_WITEcAlpha532_100x_025_10000msx1.cha"
tp = Teleport(hsinfo.cfg)
attributes = tp.parse_name(fname)
print(attributes)
_tmp = tp.cha2tmp(fname)
print(list(_tmp.keys()))

In [ ]:
for a in _tmp["root"]["raw"].attrs:
    print(a)

In [ ]:
tp = Teleport(hsinfo.cfg)
fname="D:\\nina\\DATA\\Projects\\CHARISMA\\RR1\\LBF\\SOP_WITEcAlpha532_20x_050_10000msx1_000_Spec.Data 2.cha"
#fname="D:\\nina\\DATA\\Projects\\CHARISMA\\RR1\\LBF\\S0P_WITEcAlpha532_100x_025_10000msx1.cha"
r = tp.parse_name(fname)
print(r)


In [ ]:


tp = Teleport(hsinfo.cfg)
tp.check_folder("/Round_Robin_1/inventory/")
tp.check_folder("/Round_Robin_1/inventory/participants/")
for p in partners:
    tp.check_folder("/Round_Robin_1/inventory/participants/{}/".format(p))


try:
    tp.delete_domain_recursive("/Round_Robin_1/inventory/instruments")
except:
    pass
tp.check_folder("/Round_Robin_1/inventory/instruments/")
for p in instruments:
    brand = p["brand"].replace("_"," ")
    model = p["model"].replace("_"," ")
    tp.check_folder("/Round_Robin_1/inventory/instruments/{}_{}/".format(brand,model))
    #tp.check_folder("/Round_Robin_1/inventory/instruments/{}/".format(brand))
    #tp.check_folder("/Round_Robin_1/inventory/instruments/{}/{}/".format(brand,model))
    #for w in p["wavelength"]:
    #    tp.check_folder("/Round_Robin_1/inventory/instruments/{}/{}/{}/".format(brand,model,w))


In [ ]:
try:
    tp = Teleport(hsinfo.cfg)
    tp.delete_domain_recursive("/Round_Robin_1/FraunhoferLBF")
except:
    pass

In [ ]:
cfg = hsinfo.cfg
username = cfg["hs_username"]
password = cfg["hs_password"]
endpoint = cfg["hs_endpoint"]
bucket = cfg["hs_bucket"]
if bucket==None:
    bucket="charisma"
print(bucket)

print(cfg)
path = "D:\\nina\DATA\Projects\CHARISMA\RR1\LBF"

def visitor(fname):
    print(fname)
    tp = Teleport(hsinfo.cfg)
    tp.set_domain("/Round_Robin_1/FraunhoferLBF/")
    tp.set_owner("n10705013")
    dest = tp.chada2hsds(fname,delete=False)
    return {
        'Filename': fname,
        'Domain' : dest
    }
tp = Teleport(hsinfo.cfg)    
tp.set_owner("n10705013")
tp.check_folder("/Round_Robin_1/");       
tp.check_folder("/Round_Robin_1/FraunhoferLBF/");       
r = process_file(path,recurse=True,callback=visitor)    

In [ ]:
f = h5pyd.File("/test2/silicon/test1.cha")
print(list(f.keys()))

for key in f.keys():
    print(key)
    dataset = f[key]
    for dim in dataset.dims:
        print(dim)
    print(list(dataset.attrs))
    print(dataset.shape,dataset.size,dataset.dtype,dataset.len,dataset.name)
    try:
        print(dataset[0])
    except Exception as err:
        print(err)

In [ ]:
import h5pyd._apps.hsdel as hsdel
import h5pyd._apps.hsinfo as hsinfo
print(hsinfo.cfg)
#hsdel.deleteDomain("/RR1/LBF/S0P_WITEcAlpha532_100x_005_10000msx1.cha")
hsdel.deleteDomain("/Round_Robin_1/")